In [1]:
import numpy as np
import pandas as pd
import math
np.set_printoptions(suppress=True)

In [2]:
user_artist_df = pd.read_csv('user_artist.csv')
test_df = pd.read_csv('test.csv')

In [3]:
new_df = (user_artist_df.copy())
new_test_df = (test_df.copy())
# mask = (new_test_df['userID'] == 145)
# mask1 = (new_test_df['userID'] == 145) | (new_test_df['userID'] == 1539) | (new_test_df['userID'] == 2214)| (new_test_df['userID'] == 211) | (new_test_df['userID'] == 3028)
# new_test_df = new_test_df[mask] 
new_test_df['weight'] = -1

In [4]:
new_test_df1 = pd.pivot_table(new_test_df, values = 'weight', index = ['userID'], columns = ['artistID'])

In [5]:
new_df1 = pd.pivot_table(new_df, values="weight", index=['userID'], columns=["artistID"])

In [6]:
new_df1.fillna(0, inplace=True)

In [7]:
# R avg 
r_avg = (pd.DataFrame(new_df1).to_numpy()).mean()
r_avg

1.8910221059478527

In [8]:
result = pd.concat([new_df1, new_test_df1])
# res1 = result.select_dtypes(np.number).max(level=0)
res2 = result.groupby(level = 0).median()

cols = sorted(list(res2.columns))
sortes3 = res2[cols]
sortes3.fillna(-2, inplace=True)

In [9]:
mtrx_R = pd.DataFrame(sortes3).to_numpy()

In [10]:
def calc_deviation_b(arr):
    n = len(arr[arr >= 0])
    if n == 0:
        return 0
    sum_arr = 0
    for i in range(len(arr)):
        if arr[i] >= 0:
            sum_arr += arr[i]
    return sum_arr/n

In [11]:
b_u = []
b_i = []
for i in range(len(mtrx_R)):
    b_u.append((calc_deviation_b(mtrx_R[i]) - r_avg))

mtrx_R_tr = mtrx_R.T

for i in range(len(mtrx_R_tr)):
    b_i.append((calc_deviation_b(mtrx_R_tr[i]) - r_avg))

In [12]:
# create R_hat
to_filled = []
mtrx_R_hat = np.zeros((len(mtrx_R), len(mtrx_R[0])))
for i in range(len(mtrx_R)):
     for j in range(len(mtrx_R[0])):
        # here don't use -1 yet.
        if mtrx_R[i,j] == -2:
            mtrx_R_hat[i,j] = 0
        else:
            mtrx_R_hat[i,j] = r_avg + b_u[i] + b_i[j]
            if mtrx_R[i,j] == -1:
                to_filled.append((i,j))
        
            

In [13]:
# create R_tilda
mtrx_R_tilda = mtrx_R - mtrx_R_hat

# put -1 after the sub.

for (i,j) in to_filled:
    mtrx_R_tilda[i,j] = -1


In [14]:
from sklearn.metrics.pairwise import cosine_similarity
def special_inner_profuct(mtrx): # need a faster way to calculate- 
    mtrx_transpose = mtrx.T
    D = cosine_similarity(mtrx_transpose)
#     mtrx_R_transpose_copy = mtrx_transpose.copy()
#     mtrx_R_transpose_copy = np.where(mtrx_R_transpose_copy == -1, 0, mtrx_R_transpose_copy)
#     mtrx_R_transpose_copy = np.where(mtrx_R_transpose_copy == -2, 0, mtrx_R_transpose_copy)
#     D = np.zeros((len(mtrx[0]), len(mtrx[0])))
    
#     for i in range(len(mtrx_R_transpose_copy)):
#         for j in range(len(mtrx_R_transpose_copy)):
#             mone = np.dot(mtrx_R_transpose_copy[i], mtrx_R_transpose_copy[j])
#             x = 0
#             y = 0
#             for k in range(len(mtrx_R_transpose_copy[0])):
#                 # run n^3 where n = 16548 is the number of movies.
                
#                 if mtrx_R_transpose_copy[i][k] != -1 and mtrx_R_transpose_copy[j][k] != -1 and mtrx_R_transpose_copy[i][k] != -2 and mtrx_R_transpose_copy[j][k] != -2:
#                     x += (mtrx_R_transpose_copy[i][k] ** 2)
#                     y += (mtrx_R_transpose_copy[j][k] ** 2)     
#             norm1 = x ** 0.5
#             norm2 = y ** 0.5
            
#             mechane = norm1 * norm2
#             if mechane == 0:
#                 D[i,j] = 0
#             else:
#                 D[i,j] = mone / mechane
    
#     for i in range(len(mtrx)):
#         print(mtrx[i])
#     return
            
    return D

In [15]:
def get_nearest(similarities, n):
    # n is the number of neighboors we want.
    
    sim_arr = []
    
    for i in range(len(similarities)):
        array = similarities[i].copy()
        array_non_neg = array.copy()
        for j in range(len(array)):
            if int(array_non_neg[j]) == 1 or int(array_non_neg[j]) == -1:
                array_non_neg[j] = 0
            if array_non_neg[j] < 0:
                array_non_neg[j] = array_non_neg[j] * (-1)

        ind = np.argpartition(array_non_neg, -n)[-n:]
        
        top2 = array[sorted(ind)]
        sim_arr.append((top2, sorted(ind)))

    
    return sim_arr

In [47]:
def get_final_R(mtrx_R_hat, mtrx_R_tilda, similiraties):
    print("in get_final_R")
    nearest = get_nearest(similarities, len(similarities)) # get the 2 nearest movies and their indexes.
    print("found nearest")
    
    mtrx_R_final = np.zeros((len(mtrx_R_hat), len(mtrx_R_hat[0])))
    
    print(f"need to run {len(mtrx_R_hat)} times")
    for i in range(len(mtrx_R_hat)):
        for j in range(len(mtrx_R_hat[0])):
#             if (i,j) != (0,1) and (i,j) != (1,0): # this will be replaced by the indexes we extract (cells which need to be filled)
#                 mtrx_R_final[i,j] = mtrx_R_hat[i,j] # only predict for the missing ones.
#                 continue
            
            most_similar_mvs_to_j = nearest[j][1] # similarity indexes.
            r_tilda  = (mtrx_R_tilda[i][most_similar_mvs_to_j])
            mone1 = np.dot(nearest[j][0], r_tilda)
            
            mechane1 = 0
            for k in range(len(nearest[j][0])):
                mechane1 += abs(nearest[j][0][k])
                
            if mechane1 == 0:
                mtrx_R_final[i,j] = mtrx_R_hat[i,j]
            else:
                mtrx_R_final[i,j] = mtrx_R_hat[i,j] + mone1/mechane1
        print(f"{i} iteration done")
    return mtrx_R_final

In [48]:
similarities = special_inner_profuct(mtrx_R_tilda)
print("found simliraties matrix")
m = get_final_R(mtrx_R_hat, mtrx_R_tilda, similarities)
m

found simliraties matrix
in get_final_R
found nearest
need to run 1892 times


KeyboardInterrupt: 

In [46]:
error = 0
for i in range(len(m)):
    print(f"iteration {i} out of {len(m)}")
    for j in range(len(m[0])):
        if mtrx_R[i][j] < 0:
            continue
        if mtrx_R[i][j] == 0:
            mtrx_R[i][j] = 1
            m[i][j] = 1
        if m[i][j] <= 0:
            m[i][j] = 1
        error += ((math.log(m[i][j]) - math.log(mtrx_R[i][j])) ** 2)
    print(error)
        
print(error)        

iteration 0 out of 1892
171564.99291775818
iteration 1 out of 1892
341505.73145023163
iteration 2 out of 1892
515463.5872815051
iteration 3 out of 1892
687735.3708530801
iteration 4 out of 1892
857792.1043041765
iteration 5 out of 1892
1032969.7046859593
iteration 6 out of 1892
1202879.2819643808
iteration 7 out of 1892
1375960.053080869
iteration 8 out of 1892
1548893.3519364193
iteration 9 out of 1892
1722461.6008399676
iteration 10 out of 1892
1895928.2054249095
iteration 11 out of 1892
2065727.0526788812
iteration 12 out of 1892
2237331.2623797045
iteration 13 out of 1892
2407734.717647395
iteration 14 out of 1892
2579305.189601412
iteration 15 out of 1892
2748543.032899968
iteration 16 out of 1892
2919975.5966329374
iteration 17 out of 1892
3092521.5172770796
iteration 18 out of 1892
3262895.0299008028
iteration 19 out of 1892
3438015.574613636
iteration 20 out of 1892
3612256.494245343
iteration 21 out of 1892
3786306.6091591786
iteration 22 out of 1892
3957300.3757063597
iterati

32441360.1864177
iteration 188 out of 1892
32615578.62217626
iteration 189 out of 1892
32790747.528584685
iteration 190 out of 1892
32963211.457240574
iteration 191 out of 1892
33135956.65127393
iteration 192 out of 1892
33310183.940603167
iteration 193 out of 1892
33481556.533816777
iteration 194 out of 1892
33653189.95583239
iteration 195 out of 1892
33825086.87162934
iteration 196 out of 1892
33998380.67918493
iteration 197 out of 1892
34172702.80997084
iteration 198 out of 1892
34343968.27162625
iteration 199 out of 1892
34517913.1805747
iteration 200 out of 1892
34689196.840064615
iteration 201 out of 1892
34861420.557846226
iteration 202 out of 1892
35034092.86645986
iteration 203 out of 1892
35209907.70139947
iteration 204 out of 1892
35384675.94092832
iteration 205 out of 1892
35556186.65595825
iteration 206 out of 1892
35729486.838309586
iteration 207 out of 1892
35903288.15218573
iteration 208 out of 1892
36074469.3760641
iteration 209 out of 1892
36251672.88288214
iteration 

64566497.40862622
iteration 374 out of 1892
64741691.50500402
iteration 375 out of 1892
64914456.976275675
iteration 376 out of 1892
65091974.379253626
iteration 377 out of 1892
65267365.3169656
iteration 378 out of 1892
65440754.38053123
iteration 379 out of 1892
65613340.5807544
iteration 380 out of 1892
65784572.557247
iteration 381 out of 1892
65956216.735476464
iteration 382 out of 1892
66127773.151660465
iteration 383 out of 1892
66301728.721864335
iteration 384 out of 1892
66472982.51354072
iteration 385 out of 1892
66644124.133694254
iteration 386 out of 1892
66814559.8312815
iteration 387 out of 1892
66986714.46855623
iteration 388 out of 1892
67157950.8816308
iteration 389 out of 1892
67329339.78295864
iteration 390 out of 1892
67500536.320734
iteration 391 out of 1892
67674641.80014518
iteration 392 out of 1892
67844576.47832821
iteration 393 out of 1892
68017515.96576716
iteration 394 out of 1892
68192398.31545383
iteration 395 out of 1892
68363895.86198968
iteration 396 ou

KeyboardInterrupt: 